# Splunk App for Data Science and Deep Learning - Notebook Template 

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DSDL) - formerly known as the Deep Learning Toolkit for Splunk (DLTK). Find more examples and information in the app and on the [DSDL splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

| fit MLTKContainer algo=dnn_lab num_hidden_layers=3 activation_name=ReLU dropout_rate=0.7 optimizer_name=Adam learning_rate=0.01 * into app:dnn_lab

| inputlookup dnn_lab_data.csv
| eval is_train = 100
| head 1000
| apply app:dnn_lab

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [180]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import time

# Custom Lion Optimizer
class Lion(optim.Optimizer):
    def __init__(self, params, lr=1e-4, betas=(0.9, 0.99), weight_decay=0.1):
        defaults = dict(lr=lr, betas=betas, weight_decay=weight_decay)
        super(Lion, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['exp_avg'] = torch.zeros_like(p.data)

                exp_avg = state['exp_avg']
                beta1, beta2 = group['betas']

                # Update momentum
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)

                # Compute update
                update = exp_avg.clone().mul_(beta2).add_(grad, alpha=1 - beta2).sign_()

                # Weight decay
                if group['weight_decay'] != 0:
                    update.add_(p.data, alpha=group['weight_decay'])

                # Update parameters
                p.data.add_(update, alpha=-group['lr'])

        return loss

# Define the DNN model
class SimpleDNN(nn.Module):
    def __init__(self, input_dim, num_hidden_layers, nodes_per_layer, activation_func, dropout_rate):
        super(SimpleDNN, self).__init__()
        layers = []
        # Input layer
        layers.append(nn.Linear(input_dim, nodes_per_layer))
        layers.append(activation_func)
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))

        # Hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(nodes_per_layer, nodes_per_layer))
            layers.append(activation_func)
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))

        # Output layer
        layers.append(nn.Linear(nodes_per_layer, 2))  # Binary classification
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# global constants
nodes_per_layer = 64
batch_size = 512
class_weight = 1
num_epochs = 10
MODEL_DIRECTORY = "/srv/app/model/data/"

In [181]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.26.4
pandas version: 2.2.3


To stage the data with correct default parameters use this search:

| inputlookup dnn_lab_split.csv
| fit MLTKContainer mode=stage num_hidden_layers=5 activation_name="Tanh" dropout_rate=0.0 learning_rate=0.01 optimizer_name="Adam" algo=dnn_lab * into app:dnn_lab

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("dnn_lab" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [182]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [183]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("dnn_lab")

In [184]:
df

,SS_ct_srv_src,SS_dur,SS_rate,SS_sbytes,SS_sinpkt,SS_sjit,SS_sloss,SS_spkts,SS_sttl,label,is_train
0,-0.852710,-0.188279,-0.538636,-0.047235,-0.146543,-0.054983,-0.049680,-0.087088,-0.938280,1,1
1,0.581366,-0.221547,-0.513440,-0.048464,-0.150094,-0.092147,-0.072361,-0.119896,-1.226106,0,1
2,0.171630,-0.221774,1.629959,-0.049813,-0.150149,-0.092163,-0.072361,-0.130831,0.844383,1,1
3,-0.647842,-0.094751,-0.538947,-0.047226,-0.136473,0.051424,-0.049680,-0.087088,-0.938280,0,1
4,-0.238106,-0.210221,-0.538123,-0.049294,-0.147918,-0.060849,-0.061020,-0.108960,-1.226106,0,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.033238,-0.221773,0.183947,-0.049813,-0.150148,-0.092163,-0.072361,-0.130831,0.844383,1,0
9996,-0.647842,-0.221654,-0.518401,-0.050116,-0.150148,-0.092163,-0.072361,-0.130831,-1.226106,0,0
9997,-0.033238,-0.221774,5.967993,-0.049813,-0.150149,-0.092163,-0.072361,-0.130831,0.844383,1,0
9998,-0.852710,-0.149120,-0.538817,-0.016404,-0.144734,-0.026683,-0.026999,-0.065217,-0.000525,1,0


In [185]:
param

{'options': {'params': {'mode': 'stage',
   'num_hidden_layers': '5',
   'activation_name': '"Tanh"',
   'dropout_rate': '0.0',
   'learning_rate': '0.01',
   'optimizer_name': '"Adam"',
   'algo': 'dnn_lab'},
  'args': ['*'],
  'feature_variables': ['*'],
  'model_name': 'dnn_lab',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'disabled': False,
   'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '6000',
   'max_inputs': '1000000',
   'max_memory_usage_mb': '16000',
   'max_model_size_mb': '3000',
   'max_score_time': '6000',
   'use_sampling': '1'},
  'kfold_cv': None},
 'feature_variables': ['SS_ct_srv_src',
  'SS_dur',
  'SS_rate',
  'SS_sbytes',
  'SS_sinpkt',
  'SS_sjit',
  'SS_sloss',
  'SS_spkts',
  'SS_sttl',
  'label',
  'is_train']}

## Stage 2 - create and initialize a model

In [186]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    input_dim = len(df.columns)-2 #remove training and flag field in input dimensionality
    num_hidden_layers = int(param['options']['params']['num_hidden_layers'])
    activation_name = param['options']['params']['activation_name'].strip('\"')
    # Map activation functions
    activation_mapping = {
        'ReLU': nn.ReLU(),
        'GELU': nn.GELU(),
        'Tanh': nn.Tanh()
    }
    activation_func = activation_mapping[activation_name]
    dropout_rate = float(param['options']['params']['dropout_rate'])
    
    # Convert to PyTorch tensors
    device = torch.device('cpu')
    model['num_hidden_layers'] = num_hidden_layers
    model['input_dim'] = input_dim
    model['nodes_per_layer'] = nodes_per_layer
    model['activation_name'] = activation_name
    model['dropout_rate'] = dropout_rate
    model['dnn'] = SimpleDNN(input_dim, num_hidden_layers, nodes_per_layer, activation_func, dropout_rate).to(device)
    return model

In [187]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'num_hidden_layers': 5, 'input_dim': 9, 'nodes_per_layer': 64, 'activation_name': 'Tanh', 'dropout_rate': 0.0, 'dnn': SimpleDNN(
  (layers): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): Tanh()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): Tanh()
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): Tanh()
    (10): Linear(in_features=64, out_features=2, bias=True)
  )
)}


## Stage 3 - fit the model

In [188]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    summary_list = {}
    df_train = df[df['is_train'] == 1]
    df_test = df[df['is_train'] == 0]
    X_train = df_train.drop('label', axis=1).drop('is_train', axis=1)
    X_test = df_test.drop('label', axis=1).drop('is_train', axis=1)
    y_train = df_train['label']
    y_test = df_test['label']
    
    print("\nShape of X_train:", X_train.shape)
    print("Shape of X_test:", X_test.shape)
    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)

    device = torch.device('cpu')
    # Convert pandas Series to NumPy arrays before creating tensors
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device) # Convert Series to numpy array
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)   # Convert Series to numpy array
    
    # Create TensorDataset and DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    learning_rate = float(param['options']['params']['learning_rate'])
    optimizer_name = param['options']['params']['optimizer_name'].strip('\"')

    # Calculate class weights
    total_samples = len(y_train)
    num_class_0 = np.sum(y_train == 0)
    num_class_1 = np.sum(y_train == 1)
    weight_for_class_0 = total_samples / (2.0 * num_class_0)
    weight_for_class_1 = (total_samples / (2.0 * num_class_1)) * class_weight
    class_weights = torch.tensor([weight_for_class_0, weight_for_class_1], dtype=torch.float32).to(device)

    # Create DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Define loss function with class weights
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    # Define optimizer
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model['dnn'].parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model['dnn'].parameters(), lr=learning_rate)
    elif optimizer_name == 'Lion':
        optimizer = Lion(model['dnn'].parameters(), lr=learning_rate, betas=(0.9, 0.99), weight_decay=0.1)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # Training loop
    print(f"\nTraining with: Layers={model['num_hidden_layers']}, Nodes={model['nodes_per_layer']}, LR={learning_rate}, "
          f"Batch={batch_size}, Epochs={num_epochs}, Dropout={model['dropout_rate']}, Activation={model['activation_name']}, "
          f"Optimizer={optimizer_name}, Class Weight={class_weight}")
    summary_list['training_settings'] = f"\nTraining with: Layers={model['num_hidden_layers']}, Nodes={model['nodes_per_layer']}, LR={learning_rate}, Batch={batch_size}, Epochs={num_epochs}, Dropout={model['dropout_rate']}, Activation={model['activation_name']}, Optimizer={optimizer_name}, Class Weight={class_weight}"
    start_time = time.time()
    model['dnn'].train()
    loss_list = []
    epoch_list = []
    for epoch in range(num_epochs):
        epoch_list.append(epoch+1)
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model['dnn'](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model['dnn'].parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
    
        epoch_loss = epoch_loss / len(train_dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        loss_list.append(round(epoch_loss,4))
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    summary_list['training_time'] = f"Training completed in {training_time:.2f} seconds"
    summary_list['epoch_number'] = epoch_list
    summary_list['loss_list'] = loss_list
    summary_list['final_loss'] = round(epoch_loss,4)
    with open(MODEL_DIRECTORY + "dnn_lab_loss.json", 'w') as file:
        json.dump(summary_list, file)
    
    return model

In [189]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))


Shape of X_train: (8000, 9)
Shape of X_test: (2000, 9)
Shape of y_train: (8000,)
Shape of y_test: (2000,)

Training with: Layers=5, Nodes=64, LR=0.01, Batch=512, Epochs=10, Dropout=0.0, Activation=Tanh, Optimizer=Adam, Class Weight=1
Epoch 1/10, Loss: 0.4353
Epoch 2/10, Loss: 0.3488
Epoch 3/10, Loss: 0.3216
Epoch 4/10, Loss: 0.3065
Epoch 5/10, Loss: 0.2857
Epoch 6/10, Loss: 0.2665
Epoch 7/10, Loss: 0.2406
Epoch 8/10, Loss: 0.2309
Epoch 9/10, Loss: 0.2295
Epoch 10/10, Loss: 0.2317
Training completed in 0.88 seconds
{'num_hidden_layers': 5, 'input_dim': 9, 'nodes_per_layer': 64, 'activation_name': 'Tanh', 'dropout_rate': 0.0, 'dnn': SimpleDNN(
  (layers): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): Tanh()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): Tanh()
    (8): Linear(in_fea

## Stage 4 - apply the model

In [190]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    try:
        X = df.drop('label', axis=1)
    except:
        X = df
    try:
        X = X.drop('is_train', axis=1) 
    except:
        X = df
    try:
        device = torch.device('cpu')
        X_tensor = torch.tensor(X.values, dtype=torch.float32).to(device)
        model['dnn'].eval()
        predictions = []
        with torch.no_grad(): 
            for i in range(X_tensor.shape[0]): 
                inputs = X_tensor[i:i+1] 
                output = model['dnn'](inputs)
                _, predicted = torch.max(output, 1)
                predictions.append(predicted.tolist()[0])   
        cols = {"Result": predictions}
        result = pd.DataFrame(data=cols)
    except Exception as e:
        cols = {"Error in Inference": [str(model)]}
        result = pd.DataFrame(data=cols)
    return result

In [191]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df[:100],param))

    Result
0        1
1        0
2        1
3        1
4        0
..     ...
95       0
96       1
97       0
98       0
99       0

[100 rows x 1 columns]


## Stage 5 - save the model

In [192]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    model_path = MODEL_DIRECTORY + name + ".pth"
    torch.save(model['dnn'], model_path)
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        model_files = model.copy()
        model_files.pop('dnn', None)
        json.dump(model_files, file)
    return model

In [193]:
save(model,"dnn_lab")

{'num_hidden_layers': 5,
 'input_dim': 9,
 'nodes_per_layer': 64,
 'activation_name': 'Tanh',
 'dropout_rate': 0.0,
 'dnn': SimpleDNN(
   (layers): Sequential(
     (0): Linear(in_features=9, out_features=64, bias=True)
     (1): Tanh()
     (2): Linear(in_features=64, out_features=64, bias=True)
     (3): Tanh()
     (4): Linear(in_features=64, out_features=64, bias=True)
     (5): Tanh()
     (6): Linear(in_features=64, out_features=64, bias=True)
     (7): Tanh()
     (8): Linear(in_features=64, out_features=64, bias=True)
     (9): Tanh()
     (10): Linear(in_features=64, out_features=2, bias=True)
   )
 )}

## Stage 6 - load the model

In [194]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model_params = json.load(file)

    input_dim = model_params['input_dim']
    num_hidden_layers = int(model_params['num_hidden_layers'])
    activation_name = model_params['activation_name']
    # Map activation functions
    activation_mapping = {
        'ReLU': nn.ReLU(),
        'GELU': nn.GELU(),
        'Tanh': nn.Tanh()
    }
    activation_func = activation_mapping[activation_name]
    dropout_rate = float(model_params['dropout_rate'])
    device = torch.device('cpu')
    nodes_per_layer = model_params['nodes_per_layer']

    model['dnn'] = SimpleDNN(input_dim, num_hidden_layers, nodes_per_layer, activation_func, dropout_rate).to(device)
    model_path = MODEL_DIRECTORY + name + ".pth"
    model['dnn'] = torch.load(model_path, weights_only=False)
    return model

In [195]:
model = load("dnn_lab")

In [196]:
model

{'dnn': SimpleDNN(
   (layers): Sequential(
     (0): Linear(in_features=9, out_features=64, bias=True)
     (1): Tanh()
     (2): Linear(in_features=64, out_features=64, bias=True)
     (3): Tanh()
     (4): Linear(in_features=64, out_features=64, bias=True)
     (5): Tanh()
     (6): Linear(in_features=64, out_features=64, bias=True)
     (7): Tanh()
     (8): Linear(in_features=64, out_features=64, bias=True)
     (9): Tanh()
     (10): Linear(in_features=64, out_features=2, bias=True)
   )
 )}

## Stage 7 - provide a summary of the model

In [197]:
# return a model summary
def summary(model=None):
    try:
        with open(MODEL_DIRECTORY + "dnn_lab_loss.json", 'r') as file:
            loss_info = json.load(file)
    except:
        loss_info = {'training_settings': "None", 'training_time': "None", 'epoch_number':"None", 'loss_list':"None", 'final_loss': "None"}
    returns = loss_info
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [ ]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [ ]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

In [ ]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code